In [ ]:
%pip install --upgrade pip setuptools wheel
%pip install numpy==1.23.5
%pip install tflite-runtime==2.14.0
%pip install opencv-python==4.7.0.72


: 

In [ ]:
import cv2

# Open the default camera (0 usuammlly refers to the first camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the resulting frame
    cv2.imshow('Camera', frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import mediapipe as mp

import tflite_runtime.interpreter as tflite


TARGET_SIZE = 224
MODEL_PATH = "/home/rafayahmadraza/SignBuddy/Models/asl_alphabert_model.tflite"   # your trained model file


# -------------------------------------------------------
# Resize the cropped hand to 224×224 while keeping aspect
# -------------------------------------------------------
def resize_with_padding(img, target=TARGET_SIZE):
    h, w = img.shape[:2]
    aspect = w / h

    if aspect > 1:  # wide
        new_w = target
        new_h = int(target / aspect)
    else:  # tall
        new_h = target
        new_w = int(target * aspect)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Add black padding
    top = (target - new_h) // 2
    bottom = target - new_h - top
    left = (target - new_w) // 2
    right = target - new_w - left

    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return padded


# -------------------------------------------------------
# Load TFLite Model
# -------------------------------------------------------
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model input:", input_details[0]["shape"])


# -------------------------------------------------------
# Initialize MediaPipe Hands
# -------------------------------------------------------
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)


# -------------------------------------------------------
# Main loop
# -------------------------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            # Extract bounding box
            xs = [lm.x for lm in hand_landmarks.landmark]
            ys = [lm.y for lm in hand_landmarks.landmark]

            xmin = int(min(xs) * w)
            xmax = int(max(xs) * w)
            ymin = int(min(ys) * h)
            ymax = int(max(ys) * h)

            pad = 30
            xmin = max(0, xmin - pad)
            ymin = max(0, ymin - pad)
            xmax = min(w, xmax + pad)
            ymax = min(h, ymax + pad)

            # Draw box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop
            crop = frame[ymin:ymax, xmin:xmax]

            if crop.size > 0:
                crop_224 = resize_with_padding(crop)

                # Show resized hand
                cv2.imshow("Hand 224x224", crop_224)

                # Prepare input for model
                inp = cv2.cvtColor(crop_224, cv2.COLOR_BGR2RGB)
                inp = inp.astype(np.float32) / 255.0
                inp = np.expand_dims(inp, axis=0)

                # Run inference
                interpreter.set_tensor(input_details[0]["index"], inp)
                interpreter.invoke()
                prediction = interpreter.get_tensor(output_details[0]["index"])

                # Get predicted class
                cls = np.argmax(prediction)
                conf = np.max(prediction)

                # Display prediction
                text = f"Class: {cls}  ({conf:.2f})"
                cv2.putText(frame, text, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw landmarks
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

    cv2.imshow("Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib64/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/rafayahmadraza/SignBuddy/.venv/lib64/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/rafayahmadraza/SignBuddy/.venv/lib64/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/

AttributeError: _ARRAY_API not found

SystemError: <built-in method CreateWrapperFromFile of PyCapsule object at 0x7f62f8492400> returned a result with an exception set

In [3]:
%pip uninstall -y numpy tflite-runtime tensorflow tensorflow-cpu


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: tflite-runtime 2.14.0
Uninstalling tflite-runtime-2.14.0:
  Successfully uninstalled tflite-runtime-2.14.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 96.5 kB/s  0:01:59m0:00:0300:06m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import tflite_runtime.interpreter as tflite
import numpy as np

interpreter = tflite.Interpreter(model_path="Models/asl_alphabert_model.tflite")
interpreter.allocate_tensors()

print("TFLite loaded OK!")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib64/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/rafayahmadraza/SignBuddy/.venv/lib64/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/rafayahmadraza/SignBuddy/.venv/lib64/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/rafayahmadraza/SignBuddy/.venv/lib64/pyth

AttributeError: _ARRAY_API not found

SystemError: <built-in method CreateWrapperFromFile of PyCapsule object at 0x7f62f8492400> returned a result with an exception set